In [30]:
import numpy as np
from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter

# マッピングの定義
# ファイル番号 (1-20) をエクセルのデータ書き込み開始セルにマッピング
mapping1 = {
    1: 'A8',    2: 'A13',   3: 'A18',   4: 'A24',   5: 'A28',
    6: 'A32',   7: 'A36',   8: 'A41',   9: 'A46',   10: 'A52',
    11: 'A56',  12: 'A60',  13: 'A64',  14: 'A68',  15: 'A72',
    16: 'A77',  17: 'A83',  18: 'A88',  19: 'A94',  20: 'A100'
}

# 行番号をエクセルのセルにマッピング（0になる直前の一つ前の行の値を記入）
mapping3 = {
    8: 'C4',
    13: 'D4',
    18: 'E4',
    24: 'F4',
    28: 'G4',
    32: 'H4',
    36: 'I4',
    41: 'J4',
    46: 'K4',
    52: 'L4',
    56: 'M4',
    60: 'N4',
    64: 'O4',
    68: 'P4',
    72: 'Q4',
    77: 'R4',
    83: 'S4',
    88: 'T4',
    94: 'U4',
    100: 'V4'
}

# エクセルテンプレートの読み込み
excel_template = '5.4m_original.xlsx'  # テンプレートエクセルファイルの名前を指定
try:
    wb = load_workbook(excel_template)
    ws = wb.active
except FileNotFoundError:
    print(f"エクセルテンプレートファイル '{excel_template}' が見つかりません。")
    exit(1)

# 1から20までのファイルを処理
for i in range(1, 21):
    filename = f'c_0000cm_{i:06d}.grd'  # ファイル名のフォーマット（拡張子あり）
    
    # .grdファイルの読み込み
    try:
        with open(filename, 'r') as f:
            data = []
            for line_number, line in enumerate(f, start=1):
                # 行を空白で分割し、浮動小数点数に変換
                numbers = [float(num) for num in line.strip().split()]
                if len(numbers) != 1080:
                    print(f"警告: ファイル '{filename}' の{line_number}行目に1080個の数値がありません。")
                data.append(numbers)
            data = np.array(data)
    except FileNotFoundError:
        print(f"ファイル '{filename}' が見つかりませんでした。スキップします。")
        continue
    except ValueError as ve:
        print(f"ファイル '{filename}' の読み込み中に数値変換エラーが発生しました: {ve}")
        continue
    except Exception as e:
        print(f"ファイル '{filename}' の読み込み中に予期せぬエラーが発生しました: {e}")
        continue

    num_rows, num_cols = data.shape

    if num_rows < 45:
        print(f"ファイル '{filename}' の行数が足りません: {num_rows}行しかありません。スキップします。")
        continue

    # 45行目を抽出（インデックスは0から始まるので44）
    row45 = data[44, :]  # 45行目のデータ

    # マッピング1に従ってエクセルにデータを書き出す
    cell_address = mapping1[i]
    try:
        coord = coordinate_from_string(cell_address)
        start_col_index = column_index_from_string(coord[0])  # 列インデックス (1-based)
        row = coord[1]  # 行インデックス
    except Exception as e:
        print(f"マッピング1のセルアドレス '{cell_address}' の解析中にエラーが発生しました: {e}")
        continue

    # データをエクセルに書き込む（列方向に）
    for idx, value in enumerate(row45):
        col_index = start_col_index + idx  # 書き込む列のインデックス
        if col_index > 16384:  # エクセルの最大列数 (XFD) を超えないようにする
            print(f"警告: ファイル '{filename}' のデータがエクセルの最大列数を超えています。残りのデータは書き込まれません。")
            break
        ws.cell(row=row, column=col_index, value=value)

    # 0.00000E+00になる直前の値の一つ前の行の値を特定
    zero_indices = np.where(row45 == 0)[0]
    if zero_indices.size > 0 and zero_indices[0] > 0:
        idx_before_zero = zero_indices[0] - 1
        target_col_index = start_col_index + idx_before_zero  # 対応する列インデックス
        if target_col_index < 1 or target_col_index > 16384:
            print(f"警告: ファイル '{filename}' の対象列がエクセルの範囲外です。")
            value_to_assign = None
        else:
            # 対応する列のセルの一つ前の行の値を取得
            try:
                target_col_letter = get_column_letter(target_col_index)
                source_cell = f"{target_col_letter}{row -1}"  # 一つ前の行
                value_to_assign = ws[source_cell].value
            except Exception as e:
                print(f"セル '{target_cell}' の値取得中にエラーが発生しました: {e}")
                value_to_assign = None
    else:
        # 0がない、または最初の値が0の場合
        print(f"ファイル '{filename}' の45行目に0が見つかりませんでした。最後の値を使用します。")
        value_to_assign = row45[-1]
        # 対応するセルの一つ前の行の値を取得（最後のセル）
        try:
            target_col_index = start_col_index + len(row45) -1
            if target_col_index <1 or target_col_index >16384:
                print(f"警告: ファイル '{filename}' の対象列がエクセルの範囲外です。")
                value_to_assign = None
            else:
                target_col_letter = get_column_letter(target_col_index)
                source_cell = f"{target_col_letter}{row -1}"  # 一つ前の行
                value_to_assign = ws[source_cell].value
        except Exception as e:
            print(f"セル '{source_cell}' の値取得中にエラーが発生しました: {e}")
            value_to_assign = None

    # マッピング3に従ってエクセルに値を書き出す
    target_cell = mapping3.get(row)
    if target_cell:
        if value_to_assign is not None:
            try:
                ws[target_cell] = value_to_assign
            except Exception as e:
                print(f"マッピング3のセルアドレス '{target_cell}' への書き込み中にエラーが発生しました: {e}")
        else:
            print(f"ファイル '{filename}' の値を取得できませんでした。セル '{target_cell}' への書き込みをスキップします。")
    else:
        print(f"警告: 行番号 {row} に対応するマッピング3のセルが見つかりません。値を '{target_cell}' に書き込めません。")

# エクセルファイルの保存
output_filename = 'updated_5.4m_original.xlsx'  # 保存するファイル名を指定
try:
    wb.save(output_filename)
    print(f'エクセルファイルが保存されました: {output_filename}')
except Exception as e:
    print(f"エクセルファイル '{output_filename}' の保存中にエラーが発生しました: {e}")


エクセルファイルが保存されました: updated_5.4m_original.xlsx
